In [1]:
import pandas as pd
from arcgis import GIS
gis = GIS()
import requests
import arcpy
from arcgis.features import FeatureLayer
import re

# Make Feature Layer

In [2]:
sdfCombinedForecasts = pd.DataFrame.spatial.from_featureclass('results/Traffic-Volume-Historic-and-Forecast.shp')

In [3]:
sdfCombinedForecasts = sdfCombinedForecasts[sdfCombinedForecasts['ROUTE']!='UTA']
sdfCombinedForecasts

,FID,Id,SEGID,ROUTE,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,...,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_MAXMO,FAC_MAX,FACMANADJ,SHAPE
0,0,0,0007_000.0,0007,0.000,0.076,0.080800,53.0,3.0,Dixie,...,1.0082,1.0330,1.0393,1.0366,0.9847,0.9221,4.0,1.0710,0.0,"{""paths"": [[[268173.08999999985, 4101180.67], ..."
1,1,0,0007_000.1,0007,0.076,0.297,0.225062,53.0,3.0,Dixie,...,0.9081,0.9426,1.0094,1.1264,1.1107,1.0510,10.0,1.1264,0.0,"{""paths"": [[[268284.90490000043, 4101114.28910..."
2,2,0,0007_000.3,0007,0.297,1.525,1.233974,53.0,3.0,Dixie,...,0.9081,0.9426,1.0094,1.1264,1.1107,1.0510,10.0,1.1264,0.0,"{""paths"": [[[268529.2000000002, 4100859.150000..."
3,3,0,0007_001.5,0007,1.525,2.399,0.875242,53.0,3.0,Dixie,...,0.9081,0.9426,1.0094,1.1264,1.1107,1.0510,10.0,1.1264,0.0,"{""paths"": [[[269520.8300000001, 4099147.59], [..."
4,4,0,0007_002.4,0007,2.399,3.449,1.051900,53.0,3.0,Dixie,...,0.9081,0.9426,1.0094,1.1264,1.1107,1.0510,10.0,1.1264,0.0,"{""paths"": [[[270539.79000000004, 4098210], [27..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6601,6601,0,MAG_6849,MAG,0.000,0.000,0.299553,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5.0,1.1065,0.0,"{""paths"": [[[410444.8064000001, 4459755.5428],..."
6602,6602,0,MAG_6850,MAG,0.000,0.000,0.497323,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5.0,1.1065,0.0,"{""paths"": [[[415088.00370000023, 4456424.71979..."
6603,6603,0,MAG_6851,MAG,0.000,0.000,0.516602,49.0,1.0,MAG,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5.0,1.1065,0.0,"{""paths"": [[[415170.73029999994, 4457251.986],..."
6604,6604,0,MAG_6852,MAG,0.000,0.000,2.024503,49.0,1.0,MAG,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9.0,1.0434,0.0,"{""paths"": [[[421074.06099999975, 4475350.47089..."


In [4]:
#already done in previous notebook
#sdfCombinedForecasts.spatial.to_featureclass('results/Traffic-Volume-Historic-and-Forecast.shp',sanitize_columns=False)

# Create Route Json

In [5]:
# Route Json
# R: Route Label (string, first 4 of SEGID)
# C: CO_FIPS (integer)
# label: Prefix of 
#        "I-"  + Route Label with 0s removed : for interstates
#        "Hwy" + Route Label with 0s removed : us and state routes
#        "FA"  + Route Label with 0s removed : federal aid routes
#        "WFRC/MAG/UDOT/UTA" 


dfRoutePrefix = pd.DataFrame([
    [   1,   14, 'Hwy'],
    [  15,   15, 'I-' ],
    [  16,   69, 'Hwy'],
    [  70,   70, 'I-' ],
    [  71,   79, 'Hwy'],
    [  80,   80, 'I-' ],
    [  81,   83, 'Hwy'],
    [  84,   84, 'I-' ],
    [  85,  214, 'Hwy'],
    [ 215,  215, 'I-' ],
    [ 216,  999, 'Hwy'],
    [1000, 5000, 'FA' ]
], columns=('routeFrom','routeTo','prefix'))
dfRoutePrefix

,routeFrom,routeTo,prefix
0,1,14,Hwy
1,15,15,I-
2,16,69,Hwy
3,70,70,I-
4,71,79,Hwy
5,80,80,I-
6,81,83,Hwy
7,84,84,I-
8,85,214,Hwy
9,215,215,I-


In [6]:
dfRoutes = sdfCombinedForecasts.groupby(['ROUTE','CO_FIPS'],as_index=False).agg(numSeg=('SEGID','count'))
dfRoutes['CO_FIPS'  ] = dfRoutes['CO_FIPS'].astype(int)
dfRoutes['ROUTE_numeric'] = pd.to_numeric(dfRoutes['ROUTE'], errors='coerce')

dfRoutes

,ROUTE,CO_FIPS,numSeg,ROUTE_numeric
0,0006,49,36,6.0
1,0007,53,14,7.0
2,0008,53,3,8.0
3,0009,53,30,9.0
4,0013,3,10,13.0
...,...,...,...,...
986,UDOT,53,2,NaN
987,WFRC,3,24,NaN
988,WFRC,11,47,NaN
989,WFRC,35,300,NaN


In [7]:
# Step 1: Add a dummy key to both DataFrames
dfRoutes['key'] = 1
dfRoutePrefix['key'] = 1

# Step 2: Merge on the dummy key
merged_df = pd.merge(dfRoutes, dfRoutePrefix, on='key')

# Step 3: Filter rows where ROUTE_numeric falls within the routeFrom and routeTo range
# Modify the filter to preserve NaN values (Step 3)
# We use | (OR) to add a condition that explicitly checks for NaNs in ROUTE_numeric
dfRoutePrefixWithPrefix = merged_df[((merged_df['ROUTE_numeric'] >= merged_df['routeFrom']) & 
                                     (merged_df['ROUTE_numeric'] <= merged_df['routeTo'])) |
                                     (merged_df['ROUTE_numeric'].isna())].copy()

# Fill specific columns with '' where ROUTE_numeric is NaN
# Adjust 'your_column_name_here' to the actual column(s) you want to fill for NaN ROUTE_numeric rows
dfRoutePrefixWithPrefix.loc[dfRoutePrefixWithPrefix['ROUTE_numeric'].isna(), 'prefix'   ] = dfRoutePrefixWithPrefix['ROUTE']
dfRoutePrefixWithPrefix.loc[dfRoutePrefixWithPrefix['ROUTE_numeric'].isna(), 'routeFrom'] = 0
dfRoutePrefixWithPrefix.loc[dfRoutePrefixWithPrefix['ROUTE_numeric'].isna(), 'routeTo'  ] = 0

dfRoutePrefixWithPrefix['ROUTE_numeric'].fillna(0, inplace=True)

dfRoutePrefixWithPrefix = dfRoutePrefixWithPrefix.drop_duplicates()

# Step 4: Drop the dummy key column
dfRoutePrefixWithPrefix = dfRoutePrefixWithPrefix.drop(columns=['key'])
dfRoutePrefixWithPrefix

,ROUTE,CO_FIPS,numSeg,ROUTE_numeric,routeFrom,routeTo,prefix
0,0006,49,36,6.0,1,14,Hwy
12,0007,53,14,7.0,1,14,Hwy
24,0008,53,3,8.0,1,14,Hwy
36,0009,53,30,9.0,1,14,Hwy
48,0013,3,10,13.0,1,14,Hwy
...,...,...,...,...,...,...,...
11832,UDOT,53,2,0.0,0,0,UDOT
11844,WFRC,3,24,0.0,0,0,WFRC
11856,WFRC,11,47,0.0,0,0,WFRC
11868,WFRC,35,300,0.0,0,0,WFRC


In [8]:
dfRoutePrefixWithPrefix['ROUTE_text'] = dfRoutePrefixWithPrefix['ROUTE_numeric'].apply(lambda x: str(int(x)) if pd.notnull(x) else np.nan)
dfRoutePrefixWithPrefix.loc[(dfRoutePrefixWithPrefix['ROUTE_text']=="0"), 'ROUTE_text'] = ''
dfRoutePrefixWithPrefix

,ROUTE,CO_FIPS,numSeg,ROUTE_numeric,routeFrom,routeTo,prefix,ROUTE_text
0,0006,49,36,6.0,1,14,Hwy,6
12,0007,53,14,7.0,1,14,Hwy,7
24,0008,53,3,8.0,1,14,Hwy,8
36,0009,53,30,9.0,1,14,Hwy,9
48,0013,3,10,13.0,1,14,Hwy,13
...,...,...,...,...,...,...,...,...
11832,UDOT,53,2,0.0,0,0,UDOT,
11844,WFRC,3,24,0.0,0,0,WFRC,
11856,WFRC,11,47,0.0,0,0,WFRC,
11868,WFRC,35,300,0.0,0,0,WFRC,


In [9]:
dfRoutePrefixWithPrefix.fillna('',inplace=True)
dfRoutePrefixWithPrefix['label']=dfRoutePrefixWithPrefix['prefix'] + dfRoutePrefixWithPrefix['ROUTE_text']

dfRoutePrefixWithPrefix.rename(columns={'ROUTE':'R','CO_FIPS':'C'}, inplace=True)

dfRoutePrefixWithPrefix = dfRoutePrefixWithPrefix[['R','C','label']]

dfRoutePrefixWithPrefix

,R,C,label
0,0006,49,Hwy6
12,0007,53,Hwy7
24,0008,53,Hwy8
36,0009,53,Hwy9
48,0013,3,Hwy13
...,...,...,...
11832,UDOT,53,UDOT
11844,WFRC,3,WFRC
11856,WFRC,11,WFRC
11868,WFRC,35,WFRC


In [10]:
# export to JSON
dfRoutePrefixWithPrefix.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/routes.json', orient='records', indent=4)

# Create Forecasts json

In [11]:
# Assuming sdfCombinedForecasts is your DataFrame

# Regular expression to match "F" followed by 4 digits, total length 5 characters
pattern = r'^F\d{4}$'

# Get a list of all columns that match the pattern (5 characters long: "F" + 4 digits)
columns_matching_pattern = [col for col in sdfCombinedForecasts.columns if re.match(pattern, col) and len(col) == 5]

# Display the list of columns
print(columns_matching_pattern)

# If you want to create a new DataFrame with just these columns
sdfCombinedForecasts_justneededcols = sdfCombinedForecasts[['SEGID','CO_FIPS'] + columns_matching_pattern]

# Now df_filtered_with_pattern contains only the columns from sdfCombinedForecasts that are 5 characters long, start with "F", and are followed by 4 digits
sdfCombinedForecasts_justneededcols

['F2019', 'F2023', 'F2028', 'F2032', 'F2042', 'F2050']


,SEGID,CO_FIPS,F2019,F2023,F2028,F2032,F2042,F2050
0,0007_000.0,53.0,3032.0,5800.0,5700.0,5900.0,9100.0,11000.0
1,0007_000.1,53.0,3032.0,3400.0,6800.0,16000.0,28500.0,37500.0
2,0007_000.3,53.0,4357.0,5500.0,13500.0,22500.0,42000.0,53000.0
3,0007_001.5,53.0,4357.0,4800.0,7300.0,9700.0,23000.0,28500.0
4,0007_002.4,53.0,4357.0,4800.0,7300.0,9100.0,13500.0,16500.0
...,...,...,...,...,...,...,...,...
6601,MAG_6849,49.0,0.0,0.0,0.0,0.0,0.0,5800.0
6602,MAG_6850,49.0,0.0,0.0,0.0,0.0,0.0,1200.0
6603,MAG_6851,49.0,0.0,0.0,0.0,350.0,1000.0,1000.0
6604,MAG_6852,49.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Assuming sdfCombinedForecasts_justneededcols is your DataFrame and columns_matching_pattern contains the columns to melt

# Melting the DataFrame
melted_df = pd.melt(sdfCombinedForecasts_justneededcols, id_vars=[col for col in sdfCombinedForecasts_justneededcols.columns if col not in columns_matching_pattern], 
                    value_vars=columns_matching_pattern, var_name='Year', value_name='Value')

# Extracting the year from the 'Year' column (assuming every value_var column is like 'F2021', 'F2022', etc.)
melted_df['Year'] = melted_df['Year'].apply(lambda x: x[1:]).astype(int)

melted_df['F'] = melted_df['Value'].astype(int)

melted_df['CO_FIPS'] = melted_df['CO_FIPS'].astype(int)

melted_df.drop(columns='Value', inplace=True)

melted_df.rename(columns={'SEGID':'S','Year':'Y'}, inplace=True)

# Now, melted_df contains the melted data with 'Year' as one of the columns
sdfCombinedForecasts_justneededcols_melted = melted_df[['S','Y','F','CO_FIPS']]
sdfCombinedForecasts_justneededcols_melted

,S,Y,F,CO_FIPS
0,0007_000.0,2019,3032,53
1,0007_000.1,2019,3032,53
2,0007_000.3,2019,4357,53
3,0007_001.5,2019,4357,53
4,0007_002.4,2019,4357,53
...,...,...,...,...
39301,MAG_6849,2050,5800,49
39302,MAG_6850,2050,1200,49
39303,MAG_6851,2050,1000,49
39304,MAG_6852,2050,0,49


In [13]:
# export to JSON
sdfCombinedForecasts_justneededcols_melted.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/forecasts.json', orient='records', indent=4)

# Create Observed

In [14]:
sdfCombinedForecasts.columns

Index(['FID', 'Id', 'SEGID', 'ROUTE', 'BMP', 'EMP', 'DISTANCE', 'CO_FIPS',
       'SUBAREAID', 'PLANAREA',
       ...
       'FAC_JUL', 'FAC_AUG', 'FAC_SEP', 'FAC_OCT', 'FAC_NOV', 'FAC_DEC',
       'FAC_MAXMO', 'FAC_MAX', 'FACMANADJ', 'SHAPE'],
      dtype='object', length=122)

In [15]:
# Assuming sdfCombinedForecasts is your DataFrame

# Regular expression to match "F" followed by 4 digits, total length 5 characters
pattern = r'^AADT\d{4}$'

# Get a list of all columns that match the pattern (5 characters long: "F" + 4 digits)
columns_matching_pattern_AADT = [col for col in sdfCombinedForecasts.columns if re.match(pattern, col) and len(col) == 8]

# Display the list of columns
print(columns_matching_pattern_AADT)

# If you want to create a new DataFrame with just these columns
sdfCombinedForecasts_justAADTneededcols = sdfCombinedForecasts[['SEGID'] + columns_matching_pattern_AADT]

# Now df_filtered_with_pattern contains only the columns from sdfCombinedForecasts that are 5 characters long, start with "F", and are followed by 4 digits
sdfCombinedForecasts_justAADTneededcols

['AADT2022', 'AADT2021', 'AADT2020', 'AADT2019', 'AADT2018', 'AADT2017', 'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013', 'AADT2012', 'AADT2011', 'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007', 'AADT2006', 'AADT2005', 'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001', 'AADT2000', 'AADT1999', 'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995', 'AADT1994', 'AADT1993', 'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989', 'AADT1988', 'AADT1987', 'AADT1986', 'AADT1985', 'AADT1984', 'AADT1983', 'AADT1982', 'AADT1981']


,SEGID,AADT2022,AADT2021,AADT2020,AADT2019,AADT2018,AADT2017,AADT2016,AADT2015,AADT2014,...,AADT1990,AADT1989,AADT1988,AADT1987,AADT1986,AADT1985,AADT1984,AADT1983,AADT1982,AADT1981
0,0007_000.0,4837.0,4273.0,3011.0,3032.0,2978.0,2883.0,1457.0,1380.0,1320.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0007_000.1,4837.0,4273.0,3011.0,3032.0,2978.0,2883.0,1457.0,1380.0,1320.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0007_000.3,8545.0,7549.0,4327.0,4357.0,3612.0,2883.0,2210.0,1700.0,1445.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0007_001.5,8545.0,7549.0,4327.0,4357.0,3612.0,2883.0,2210.0,1700.0,1445.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0007_002.4,8545.0,7549.0,4327.0,4357.0,3612.0,2883.0,2210.0,1700.0,1445.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6601,MAG_6849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6602,MAG_6850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6603,MAG_6851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6604,MAG_6852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
sdfCombinedForecasts_justAADTneededcols.columns

Index(['SEGID', 'AADT2022', 'AADT2021', 'AADT2020', 'AADT2019', 'AADT2018',
       'AADT2017', 'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013', 'AADT2012',
       'AADT2011', 'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007', 'AADT2006',
       'AADT2005', 'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001', 'AADT2000',
       'AADT1999', 'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995', 'AADT1994',
       'AADT1993', 'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989', 'AADT1988',
       'AADT1987', 'AADT1986', 'AADT1985', 'AADT1984', 'AADT1983', 'AADT1982',
       'AADT1981'],
      dtype='object')

In [17]:
# Assuming sdfCombinedForecasts_justneededcols is your DataFrame and columns_matching_pattern contains the columns to melt

# Melting the DataFrame
melted_df = pd.melt(sdfCombinedForecasts_justAADTneededcols, id_vars=[col for col in sdfCombinedForecasts_justAADTneededcols.columns if col not in columns_matching_pattern_AADT], 
                    value_vars=columns_matching_pattern_AADT, var_name='Year', value_name='Value')

# Extracting the year from the 'Year' column (assuming every value_var column is like 'F2021', 'F2022', etc.)
melted_df['Year'] = melted_df['Year'].apply(lambda x: x[4:]).astype(int)

melted_df = melted_df[melted_df['Year']>=2000]

melted_df['O'] = melted_df['Value'].astype(int)

melted_df.drop(columns='Value', inplace=True)

melted_df.rename(columns={'SEGID':'S','Year':'Y'}, inplace=True)

# Now, melted_df contains the melted data with 'Year' as one of the columns
sdfCombinedForecasts_justAADTneededcols_melted = melted_df[['S','Y','O']]
sdfCombinedForecasts_justAADTneededcols_melted

,S,Y,O
0,0007_000.0,2022,4837
1,0007_000.1,2022,4837
2,0007_000.3,2022,8545
3,0007_001.5,2022,8545
4,0007_002.4,2022,8545
...,...,...,...
150668,MAG_6849,2000,0
150669,MAG_6850,2000,0
150670,MAG_6851,2000,0
150671,MAG_6852,2000,0


In [18]:
sdfCombinedForecasts_justAADTneededcols_melted[sdfCombinedForecasts_justAADTneededcols_melted['S']=='0015_000.0']

,S,Y,O
54,0015_000.0,2022,30108
6605,0015_000.0,2021,29259
13156,0015_000.0,2020,23546
19707,0015_000.0,2019,24681
26258,0015_000.0,2018,24316
32809,0015_000.0,2017,23224
39360,0015_000.0,2016,21992
45911,0015_000.0,2015,20195
52462,0015_000.0,2014,18910
59013,0015_000.0,2013,18630


In [19]:
# export to JSON
sdfCombinedForecasts_justAADTneededcols_melted.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/observed.json', orient='records', indent=0)

# create segments json

In [20]:
dfSegments = pd.merge(sdfCombinedForecasts[['SEGID','ROUTE','BMP']],dfRoutePrefixWithPrefix, left_on='ROUTE', right_on='R')
dfSegments.rename(columns={'SEGID':'S','label':'N'},inplace=True)

# set label
dfSegments['L'] = dfSegments['N'] + ' ~MP' + dfSegments['BMP'].round(1).astype(str)
dfSegments.loc[(dfSegments['R'].isin(['WFRC','MAG','CACHE','DIXIE','UDOT','UTA'])), 'L'] = dfSegments['S']


dfSegments.drop(columns=['ROUTE','BMP'],inplace=True)
dfSegments

,S,R,C,N,L
0,0007_000.0,0007,53,Hwy7,Hwy7 ~MP0.0
1,0007_000.1,0007,53,Hwy7,Hwy7 ~MP0.1
2,0007_000.3,0007,53,Hwy7,Hwy7 ~MP0.3
3,0007_001.5,0007,53,Hwy7,Hwy7 ~MP1.5
4,0007_002.4,0007,53,Hwy7,Hwy7 ~MP2.4
...,...,...,...,...,...
10759,MAG_6851,MAG,49,MAG,MAG_6851
10760,MAG_6852,MAG,23,MAG,MAG_6852
10761,MAG_6852,MAG,49,MAG,MAG_6852
10762,MAG_6853,MAG,23,MAG,MAG_6853


In [21]:
# export to JSON
dfSegments.to_json('_site-traffic-volume-map/widgets/ForecastSidebar/data/segments.json', orient='records', indent=0)

In [22]:
dfSegments[dfSegments['C']==21]

,S,R,C,N,L
47,0014_000.0,0014,21,Hwy14,Hwy14 ~MP0.0
48,0014_000.3,0014,21,Hwy14,Hwy14 ~MP0.3
49,0014_000.4,0014,21,Hwy14,Hwy14 ~MP0.4
50,0014_000.5,0014,21,Hwy14,Hwy14 ~MP0.5
51,0014_001.0,0014,21,Hwy14,Hwy14 ~MP1.0
...,...,...,...,...,...
3364,UDOT_7000,UDOT,21,UDOT,UDOT_7000
3366,UDOT_7001,UDOT,21,UDOT,UDOT_7001
3368,UDOT_7006,UDOT,21,UDOT,UDOT_7006
3370,UDOT_7007,UDOT,21,UDOT,UDOT_7007
